# 1. Teams Filtering

The link to all these APIs is: https://www.game-change.co.uk/2023/02/10/a-complete-guide-to-the-fantasy-premier-league-fpl-api/

Descriptions of each dataframe are in the notes.

In [ ]:
#Importing libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

print("Libraries imported successfully......")

### i. Add fixture data to the teams data

a) Teams DF

In [ ]:
# Imported full data from last seasons

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()


#Convert to df

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [ ]:
elements_df = elements_df.rename(columns={'element_type': 'Position'})


In [ ]:
def get_position(Position):
    for idx, row in elements_types_df.iterrows():
        if Position == row['id']:
          return row['plural_name_short']
    return None  # Return None if no matching fixture is found

elements_df['Position'] = elements_df['Position'].apply(get_position)


In [ ]:
def append_event_id(teams_df):
    # Create an empty DataFrame to store the expanded data
    expanded_df = pd.DataFrame()

    # Iterate through events from 1 to 38
    for event_id in range(1, 39):
        # Create a temporary DataFrame for the current event
        temp_df = teams_df.copy()

        # Set the 'event_id' column to the current event ID
        temp_df['event_id'] = event_id

        # Append the temporary DataFrame to the expanded DataFrame
        expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)

    return expanded_df

# Call the function to create an expanded DataFrame with 'event_id'
expanded_teams_df = append_event_id(teams_df)


b) Fixture DF

In [ ]:
import requests
import pandas as pd

def fetch_fixture_data(event_id):
    # Define the base URL with the event_id
    base_url = f'https://fantasy.premierleague.com/api/fixtures/?event={event_id}'

    # Replace the placeholder '{}' with the actual event_id
    formatted_url = base_url.format(event_id)

    # Send an HTTP GET request to the API endpoint
    response = requests.get(formatted_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response to get fixture data
        fixture_json = response.json()

        # Create a DataFrame from the fixture data
        fixture_data = pd.DataFrame(fixture_json)

        return fixture_data
    else:
        print(f"Request failed for event {event_id} with status code {response.status_code}")
        return None  # Return None in case of a failed request


In [ ]:
# Create an empty DataFrame to store the expanded fixture data
expanded_fixture_df = pd.DataFrame()

# Iterate through events from 1 to 38
for event_id in range(1, 39):
# Get fixture data for the current event
    fixture_data = fetch_fixture_data(event_id)

    # Create a temporary DataFrame for the fixture data
    temp_df = fixture_data.copy()

    # Set the 'event_id' column to the current event ID
    temp_df['event_id'] = event_id

    # Append the temporary DataFrame to the expanded fixture DataFrame
    expanded_fixture_df = pd.concat([expanded_fixture_df, temp_df], ignore_index=True)



In [ ]:
# Renaming columns
expanded_teams_df = expanded_teams_df.rename(columns={'name' : 'team_name', 'id':'team_id', 'short_name' : 'team_short_name'})
expanded_fixture_df = expanded_fixture_df.rename(columns={'id':'fixture_id'})

#### i.i) Get fixture id

In [ ]:
def get_fixture_id(event_id,team_id):
    for idx, row in expanded_fixture_df.iterrows():
        if event_id == row['event_id']:
            if team_id == row['team_a'] or team_id == row['team_h']:
                return row['fixture_id']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture IDs for all teams in teams_df
expanded_teams_df['fixture_id'] = expanded_teams_df.apply(lambda row: get_fixture_id(row['event_id'], row['team_id']), axis=1)


#### i.ii) Get fixture difficulty



In [ ]:
def get_fixture_difficulty(team_id, fixture_id):
    for idx, row in expanded_fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            if team_id == row['team_a']:
                return row['team_a_difficulty']

            elif team_id == row['team_h']:
                return row['team_h_difficulty']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
expanded_teams_df['fixture_difficulty'] = expanded_teams_df.apply(lambda row: get_fixture_difficulty(row['team_id'], row['fixture_id']), axis=1)


#### i.iii) Team and opponent score

In [ ]:
def get_opponent_team(team_id, fixture_id):
    for idx, row in expanded_fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            if team_id == row['team_a']:
                return row['team_h']

            elif team_id == row['team_h']:
                return row['team_a']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
expanded_teams_df['opponent_team'] = expanded_teams_df.apply(lambda row: get_opponent_team(row['team_id'], row['fixture_id']), axis=1)


In [ ]:
def get_opponent_score(team_id, fixture_id):
    for idx, row in expanded_fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            if team_id == row['team_a']:
                return row['team_h_score']

            elif team_id == row['team_h']:
                return row['team_a_score']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
expanded_teams_df['opponent_score'] = expanded_teams_df.apply(lambda row: get_opponent_score(row['team_id'], row['fixture_id']), axis=1)


In [ ]:
def get_team_score(team_id, fixture_id):
    for idx, row in expanded_fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            if team_id == row['team_a']:
                return row['team_a_score']

            elif team_id == row['team_h']:
                return row['team_h_score']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
expanded_teams_df['team_score'] = expanded_teams_df.apply(lambda row: get_team_score(row['team_id'], row['fixture_id']), axis=1)


In [ ]:
def get_finished_status(team_id, fixture_id):
    for idx, row in expanded_fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            return row['finished']



# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
expanded_teams_df['Game_played'] = expanded_teams_df.apply(lambda row: get_finished_status(row['team_id'], row['fixture_id']), axis=1)

#### i.iv) If fixture is home

In [ ]:
def get_home_status(team_id, fixture_id):
    for idx, row in expanded_fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            if team_id == row['team_a']:
                return False

            elif team_id == row['team_h']:
                return True
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
expanded_teams_df['is_home'] = expanded_teams_df.apply(lambda row: get_home_status(row['team_id'], row['fixture_id']), axis=1)


#### i.v) If deadline passed

In [ ]:
def get_kickoff_time(team_id, fixture_id):
    for idx, row in expanded_fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            return row['kickoff_time']



# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
expanded_teams_df['kickoff_time'] = expanded_teams_df.apply(lambda row: get_kickoff_time(row['team_id'], row['fixture_id']), axis=1)

In [ ]:
def started(team_id, fixture_id):
    for idx, row in expanded_fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            return row['started']



# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
expanded_teams_df['started'] = expanded_teams_df.apply(lambda row: started(row['team_id'], row['fixture_id']), axis=1)

Gameweek Deadline variable

In [ ]:
expanded_teams_df['kickoff_time'] = pd.to_datetime(expanded_teams_df['kickoff_time'])


In [ ]:
from datetime import timedelta


# Filter the DataFrame to get events where 'started' is False
filtered_events = expanded_teams_df[expanded_teams_df['started'] == False]

# Sort the filtered events by 'kickoff_time' in ascending order
filtered_events = filtered_events.sort_values(by='kickoff_time')

# Get the first event in the sorted list
first_event = filtered_events.iloc[0]

# Calculate an hour before the 'kickoff_time' of the first event
Gameweek_Deadline = first_event['kickoff_time'] - timedelta(hours=1)

# You can now use Gameweek_Deadline as your constant
print("Gameweek_Deadline:", Gameweek_Deadline)

### ii. Feature Engineering for team data

In [ ]:
# List of columns to drop
columns_to_drop = ['played', 'points', 'position','strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away','draw', 'form','strength','win','loss','team_division']

# Use the drop method to remove the specified columns
expanded_teams_df.drop(columns=columns_to_drop, inplace=True)


In [ ]:
expanded_teams_df = expanded_teams_df.sort_values(by='kickoff_time')


In [ ]:
teams = expanded_teams_df


### General Stats

Games played


In [ ]:
# Convert 'Game_played' column to integers (1 for True, 0 for False)
teams['Game_played'].fillna(False, inplace=True)
teams['Game_played'] = teams['Game_played'].astype(int)

# Calculate cumulative 'Games_Count' for each row
teams['Games_Count'] = teams.groupby(['team_id', 'team_name'])['Game_played'].cumsum()



Total goals

In [ ]:
teams['Total_Goals'] = teams.groupby(['team_id', 'team_name'])['team_score'].cumsum()


Goals per game

In [ ]:
teams['Goals_per_game'] = teams['Total_Goals'] / teams['Games_Count']

Goals conceded

In [ ]:
teams['Total_Conceded'] = teams.groupby(['team_id', 'team_name'])['opponent_score'].cumsum()


In [ ]:
teams['Conceded_per_game'] = teams['Total_Conceded'] / teams['Games_Count']

Win

In [ ]:
teams['Win'] = teams['team_score'] > teams['opponent_score']
teams['Win'] = teams['Win'].astype(int)


Draw

In [ ]:
teams['Draw'] = teams['team_score'] == teams['opponent_score']
teams['Draw'] = teams['Draw'].astype(int)

Loss

In [ ]:
teams['Loss'] = teams['team_score'] < teams['opponent_score']
teams['Loss'] = teams['Loss'].astype(int)

Total wins

In [ ]:
teams['Total_wins'] = teams.groupby(['team_id', 'team_name'])['Win'].cumsum()


Win percentage

In [ ]:
teams['Win_percentage'] = (teams['Total_wins'] / teams['Games_Count']) * 100

Game Results

In [ ]:
teams['result'] = teams.apply(lambda row: 1 if row['Win'] else (-1 if row['Loss'] else 0), axis=1)


Form

In [ ]:
def calculate_form(group, num_games):
    forms = []
    current_form = ""

    for i, row in group.iterrows():
        result = row['result']
        if row['Game_played'] == True:
            if result == 1:
                current_form += 'W'
            elif result == -1:
                current_form += 'L'
            else:
                current_form += 'D'

            if len(current_form) > num_games:
                current_form = current_form[1:]

            forms.append(current_form)

        else:
            forms.append(None)
    return forms

# Calculate the form for each specific 'team_id' and assign it to the 'Form' column for that team
unique_team_ids = teams['team_id'].unique()
for team_id in unique_team_ids:
    team_subset = teams[teams['team_id'] == team_id]
    team_forms = calculate_form(team_subset, 5)
    teams.loc[teams['team_id'] == team_id, 'Form'] = team_forms




Numeric Form

In [ ]:
def calculate_numerical_form(group, num_games):
    numerical_forms = []
    current_form = []

    for i, row in group.iterrows():
        result = row['result']
        game_played = row['Game_played']

        if game_played:
            # Assign weights based on the position in the form
            weight = 1 + i / num_games

            if result == 1:
                value = 1
            elif result == -1:
                value = -1
            else:
                value = 0

            current_form.append(value * weight)

            if len(current_form) > num_games:
                current_form.pop(0)

            numerical_form = sum(current_form) / len(current_form)
            numerical_forms.append(numerical_form)

        else:
            numerical_forms.append(None)  # Append None for games that haven't been played

    return numerical_forms

# Calculate the numerical form for each specific 'team_id' and assign it to the 'Numerical_Form' column for that team
unique_team_ids = teams['team_id'].unique()
for team_id in unique_team_ids:
    team_subset = teams[teams['team_id'] == team_id]
    team_numerical_forms = calculate_numerical_form(team_subset, 5)
    teams.loc[teams['team_id'] == team_id, 'Numerical_Form'] = team_numerical_forms



Team strength

In [ ]:
teams['Strength'] = teams['Numerical_Form']*2.5 + teams['fixture_difficulty']*2.0 + teams['Win_percentage']*2.0 + teams['Goals_per_game'] + teams['Conceded_per_game']*-1.0

In [ ]:
Liverpool = teams[teams['team_id'] == 11]
Liverpool

### Home Stats

Games played home

In [ ]:
# Calculate cumulative 'Games_Count' for each row
teams['Home_Count'] = teams[teams['is_home'] == True].groupby(['team_id', 'team_name'])['Game_played'].cumsum()



Goals at home

In [ ]:
teams['Home_Goals'] = teams[teams['is_home'] == True].groupby(['team_id', 'team_name'])['team_score'].cumsum()


Goals per home game

In [ ]:
teams['Goals_per_home'] = teams.apply(lambda row: row['Total_Goals'] / row['Home_Count'] if row['is_home'] else None, axis=1)


Goals conceded at home

In [ ]:
teams['Home_Conceded'] = teams[teams['is_home'] == True].groupby(['team_id', 'team_name'])['opponent_score'].cumsum()


Conceded per home game

In [ ]:
teams['Conceded_per_home'] = teams.apply(lambda row: row['Home_Conceded'] / row['Home_Count'] if row['is_home'] else None, axis=1)


Total home wins

In [ ]:
teams['Total_Home_wins'] = teams[teams['is_home'] == True].groupby(['team_id', 'team_name'])['Win'].cumsum()


Home Win percentage

In [ ]:
teams['Home_Win_percentage'] = (teams['Total_Home_wins'] / teams['Home_Count']) * 100

Home Form

In [ ]:
def calculate_form_home(group, num_games):
    forms = []
    current_form = ""

    for i, row in group.iterrows():
        result = row['result']
        if row['Game_played'] == True and row['is_home'] == True:
            if result == 1:
                current_form += 'W'
            elif result == -1:
                current_form += 'L'
            else:
                current_form += 'D'

            if len(current_form) > num_games:
                current_form = current_form[1:]

            forms.append(current_form)

        else:
            forms.append(None)
    return forms

# Calculate the form for each specific 'team_id' and assign it to the 'Form' column for that team
unique_team_ids = teams['team_id'].unique()
for team_id in unique_team_ids:
    team_subset = teams[teams['team_id'] == team_id]
    team_forms = calculate_form_home(team_subset, 5)
    teams.loc[teams['team_id'] == team_id, 'Home_Form'] = team_forms




Home Numeric Form

In [ ]:
def calculate_numerical_form_home(group, num_games):
    numerical_forms = []
    current_form = []

    for i, row in group.iterrows():
        result = row['result']
        game_played = row['Game_played']

        if game_played and row['is_home'] == True:
            # Assign weights based on the position in the form
            weight = 1 + i / num_games

            if result == 1:
                value = 1
            elif result == -1:
                value = -1
            else:
                value = 0

            current_form.append(value * weight)

            if len(current_form) > num_games:
                current_form.pop(0)

            numerical_form = sum(current_form) / len(current_form)
            numerical_forms.append(numerical_form)

        else:
            numerical_forms.append(None)  # Append None for games that haven't been played

    return numerical_forms

# Calculate the numerical form for each specific 'team_id' and assign it to the 'Numerical_Form' column for that team
unique_team_ids = teams['team_id'].unique()
for team_id in unique_team_ids:
    team_subset = teams[teams['team_id'] == team_id]
    team_numerical_forms = calculate_numerical_form_home(team_subset, 5)
    teams.loc[teams['team_id'] == team_id, 'Home_Numerical_Form'] = team_numerical_forms



In [ ]:
teams['Home_Strength'] = teams['Home_Numerical_Form']*2.5 + teams['fixture_difficulty']*2.0 + teams['Home_Win_percentage']*2.0 + teams['Goals_per_home'] + teams['Conceded_per_home']*-1.0

### Away Stats

Games played away

In [ ]:
# Calculate cumulative 'Games_Count' for each row
teams['Away_Count'] = teams[teams['is_home'] == False].groupby(['team_id', 'team_name'])['Game_played'].cumsum()



Goals away

In [ ]:
teams['Away_Goals'] = teams[teams['is_home'] == False].groupby(['team_id', 'team_name'])['team_score'].cumsum()


Goals per away game

In [ ]:
teams['Goals_per_away'] = teams.apply(lambda row: row['Total_Goals'] / row['Away_Count'] if row['is_home']==False else None, axis=1)


Goals conceded away

In [ ]:
teams['Away_Conceded'] = teams[teams['is_home'] == False].groupby(['team_id', 'team_name'])['opponent_score'].cumsum()


Conceded per away game

In [ ]:
teams['Conceded_per_away'] = teams.apply(lambda row: row['Away_Conceded'] / row['Away_Count'] if row['is_home'] ==False else None, axis=1)


Total Away wins

In [ ]:
teams['Total_Away_wins'] = teams[teams['is_home'] == False].groupby(['team_id', 'team_name'])['Win'].cumsum()


Away Win percentage

In [ ]:
teams['Away_Win_percentage'] = (teams['Total_Away_wins'] / teams['Away_Count']) * 100

Away Form

In [ ]:
def calculate_form_away(group, num_games):
    forms = []
    current_form = ""

    for i, row in group.iterrows():
        result = row['result']
        if row['Game_played'] == True and row['is_home'] == False:
            if result == 1:
                current_form += 'W'
            elif result == -1:
                current_form += 'L'
            else:
                current_form += 'D'

            if len(current_form) > num_games:
                current_form = current_form[1:]

            forms.append(current_form)

        else:
            forms.append(None)
    return forms

# Calculate the form for each specific 'team_id' and assign it to the 'Form' column for that team
unique_team_ids = teams['team_id'].unique()
for team_id in unique_team_ids:
    team_subset = teams[teams['team_id'] == team_id]
    team_forms = calculate_form_away(team_subset, 5)
    teams.loc[teams['team_id'] == team_id, 'Away_Form'] = team_forms




Away Numeric Form

In [ ]:
def calculate_numerical_form_away(group, num_games):
    numerical_forms = []
    current_form = []

    for i, row in group.iterrows():
        result = row['result']
        game_played = row['Game_played']

        if game_played and row['is_home'] == False:
            # Assign weights based on the position in the form
            weight = 1 + i / num_games

            if result == 1:
                value = 1
            elif result == -1:
                value = -1
            else:
                value = 0

            current_form.append(value * weight)

            if len(current_form) > num_games:
                current_form.pop(0)

            numerical_form = sum(current_form) / len(current_form)
            numerical_forms.append(numerical_form)

        else:
            numerical_forms.append(None)  # Append None for games that haven't been played

    return numerical_forms

# Calculate the numerical form for each specific 'team_id' and assign it to the 'Numerical_Form' column for that team
unique_team_ids = teams['team_id'].unique()
for team_id in unique_team_ids:
    team_subset = teams[teams['team_id'] == team_id]
    team_numerical_forms = calculate_numerical_form_away(team_subset, 5)
    teams.loc[teams['team_id'] == team_id, 'Away_Numerical_Form'] = team_numerical_forms



In [ ]:
teams['Away_Strength'] = teams['Away_Numerical_Form']*2.5 + teams['fixture_difficulty']*2.0 + teams['Away_Win_percentage']*2.0 + teams['Goals_per_away'] + teams['Conceded_per_away']*-1.0

## Defense Strength stats

### Overall

In [ ]:
teams['Defence_Strength'] = teams['Numerical_Form']*2.0 + teams['fixture_difficulty']*1.5 + teams['Conceded_per_game']*-1.5 + teams['Win_percentage']*1.5

Home

In [ ]:
teams['Home_Defence_Strength'] = teams['Home_Numerical_Form']*2.0 + teams['fixture_difficulty']*1.5 + teams['Conceded_per_home']*-1.5 + teams['Home_Win_percentage']*1.5

Away

In [ ]:
teams['Away_Defence_Strength'] = teams['Away_Numerical_Form']*2.0 + teams['fixture_difficulty']*1.5 + teams['Conceded_per_away']*-1.5 + teams['Away_Win_percentage']*1.5

## Attack Strength stats

### Overall

In [ ]:
teams['Attack_Strength'] = teams['Numerical_Form']*2.5 + teams['fixture_difficulty']*2.0 + teams['Win_percentage']*0.8 + teams['Goals_per_game']*2.0

Home

In [ ]:
teams['Home_Attack_Strength'] = teams['Home_Numerical_Form']*2.5 + teams['fixture_difficulty']*2.0 + teams['Home_Win_percentage']*0.8 + teams['Goals_per_home']*2.0

Away

In [ ]:
teams['Away_Attack_Strength'] = teams['Away_Numerical_Form']*2.5 + teams['fixture_difficulty']*2.0 + teams['Away_Win_percentage']*0.8 + teams['Goals_per_away']*2.0

## Preprocessing

In [ ]:
teams.columns

In [ ]:
num_cols_normalize = ['Numerical_Form', 'Home_Numerical_Form','Away_Numerical_Form', 'Strength', 'Home_Strength', 'Away_Strength', 'Defence_Strength', 'Home_Defence_Strength', 'Away_Defence_Strength', 'Away_Attack_Strength', 'Home_Attack_Strength', 'Attack_Strength']

In [ ]:
from sklearn.preprocessing import MinMaxScaler


# Fill null values with previous values
teams[num_cols_normalize] = teams[num_cols_normalize]
# Initialize the Min-Max Scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns to scale them between 0 and 1
teams[num_cols_normalize] = scaler.fit_transform(teams[num_cols_normalize])



In [ ]:
teams.columns

In [ ]:
filtered_teams = teams.loc[:, ['team_id', 'team_name', 'team_short_name',
    'unavailable','event_id', 'fixture_id', 'fixture_difficulty','is_home','kickoff_time', 'started',
    'Attack_Strength','team_score',
    'Form', 'Numerical_Form','Win_percentage', 'Strength', 'Defence_Strength',
    'Home_Form','Home_Numerical_Form','Home_Win_percentage','Home_Strength', 'Home_Defence_Strength', 'Home_Attack_Strength',
    'Away_Form', 'Away_Numerical_Form', 'Away_Win_percentage','Away_Strength', 'Away_Defence_Strength', 'Away_Attack_Strength'
    ]]

# 2. Manager data

In [ ]:
import pandas as pd
import requests

# Manager ID and Gameweek
manager_id = 1873280
GW = 8
# API URL to get the manager's team for a specific Gameweek
url = f"https://fantasy.premierleague.com/api/entry/{manager_id}/event/{GW}/picks/"

# Make the API request to get the manager's team
r = requests.get(url)
data = r.json()

# Create a list of dictionaries to store 'picks' along with metadata
picks_data = data.get('picks', [])  # Ensure 'picks' data is available

# Extract metadata
active_chip = data.get('active_chip')
automatic_subs = data.get('automatic_subs')
entry_history = data.get('entry_history')

# Create a list with metadata for each 'picks' element
data_list = []
for pick in picks_data:
    data_list.append({
        'active_chip': active_chip,
        'automatic_subs': automatic_subs,
        'entry_history': entry_history,
        **pick,  # Include the 'picks' data
    })

# Convert the list of dictionaries to a DataFrame
manager = pd.DataFrame(data_list)


In [ ]:
manager

In [ ]:
manager.columns

In [ ]:
FPL_team = manager.loc[:,['active_chip', 'automatic_subs', 'element', 'position',
       'multiplier', 'is_captain', 'is_vice_captain']]

# 3. Model Training

## i. Player data Current Season


In [ ]:
### Function to return player url
def format_player_url(player_id):
    # Define the base URL without curly brackets
    base_url = 'https://fantasy.premierleague.com/api/element-summary/{}/'

    # Replace the placeholder '{}' with the actual player_id
    formatted_url = base_url.format(player_id)

    return formatted_url

# Example usage:
player_id = 447  # Replace this with the actual player ID
formatted_url = format_player_url(player_id)
print(formatted_url)


In [ ]:
# Imported full data from last seasons


elements = elements_df.loc[:,['id','team','web_name','first_name','second_name','Position']]
elements = elements.rename(columns = {'id': 'player_id', 'team':'team_id'})
# List of columns to drop



## ii. Feature Engineering players

Features_engineered
>['Minutes_per_game', 'Player_Strength', 'XA', 'XG', 'XS']

intended_features
>['fixture_difficulty','kickoff_time', 'started',
       'is_home', 'Attack_Strength', 'team_score', 'Form', 'Numerical_Form',
       'Win_percentage', 'Strength', 'Defence_Strength', 'Home_Form',
       'Home_Numerical_Form', 'Home_Win_percentage', 'Home_Strength',
       'Home_Defence_Strength', 'Home_Attack_Strength', 'Away_Form',
       'Away_Numerical_Form', 'Away_Win_percentage', 'Away_Strength',
       'Away_Defence_Strength', 'Away_Attack_Strength']

intended_labels
>['goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'saves']

## iii. Model Selection

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import requests
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression

num_top_features = 8  # Set it to the total number of features

def load_data_for_player(player_id):
    # Define the URL for the player's data
    player_url = format_player_url(player_id)  # You can use the provided function

    # Send a GET request to the player's URL
    response = requests.get(player_url)

    if response.status_code == 200:
        # Parse the JSON data
        player_data = response.json()

        # Extract the relevant data and preprocess it
        player_history_df = pd.DataFrame(player_data['history'])

        player_history_df = player_history_df.rename(columns={'element': 'player_id', 'round': 'event_id'})
        columns_to_drop = ['was_home', 'bps', 'kickoff_time', 'influence', 'creativity', 'threat', 'ict_index', 'starts',
                           'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded',
                           'value', 'transfers_balance', 'selected', 'transfers_in', 'transfers_out', 'fixture',
                           'team_h_score', 'team_a_score', 'event_id']

        # Use the drop method to remove the specified columns
        player_history_df.drop(columns=columns_to_drop, inplace=True)
        player_history_df = player_history_df.merge(elements, on='player_id')
        player_history_df = player_history_df.merge(filtered_teams, on=['team_id'])
        player_history_df['player_fixture_id'] = player_history_df['player_id'].astype(str) + '_' + player_history_df['event_id'].astype(str)

        # Perform any necessary data cleaning and feature engineering here
        player_history_df['kickoff_time'] = pd.to_datetime(player_history_df['kickoff_time'])
        player_history_df.set_index('kickoff_time', inplace=True)
        player_history_df = player_history_df.fillna(method='ffill')
        player_history_df['is_home'] = player_history_df['is_home'].astype(int)
        labels = ['goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'saves']

        player_future_fxt = player_history_df[player_history_df['started'] == False].head(3)
        player_future_fxt.drop(columns=labels, inplace=True)
        player_history_df = player_history_df[player_history_df['started'] == True]

        # Return the processed data for the player
        return player_history_df, player_future_fxt
    else:
        # Handle the case when the request fails (e.g., return None or raise an exception)
        return None, None

num_top_features = 8  # Set it to the total number of features

def train_model(X_train, y_train):
    # Define a scikit-learn pipeline
    pipeline = Pipeline([
        ('feature_selector', SelectKBest(score_func=f_regression, k=num_top_features)),
        ('scaler', StandardScaler(with_mean=False)),
        ('model', tf.keras.Sequential([
            tf.keras.layers.Input(shape=(num_top_features,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(1)  # Output layer with 1 neuron for regression
        ]))
    ])

    # Compile the model within the pipeline with run_eagerly=True
    pipeline.named_steps['model'].compile(optimizer='adam', loss='mean_squared_error', run_eagerly=True)

    # Train the model within the pipeline on the provided training data
    pipeline.named_steps['model'].fit(X_train, y_train, epochs=100, batch_size=32)

    return pipeline

# Define a function to process a player
def process_player(player_id):
    Don_Team = pd.DataFrame()
    Don_Team_list = []    # Load data for the specified player using player_id
    player_history_df, player_future_fxt = load_data_for_player(player_id)

    if player_history_df is not None and player_future_fxt is not None:
        # Define features based on the data for the player
        features = ['fixture_difficulty', 'is_home', 'Attack_Strength', 'team_score', 'Numerical_Form', 'Win_percentage',
                    'Strength', 'Defence_Strength']
        labels = ['goals_scored', 'assists', 'clean_sheets', 'goals_conceded','bonus', 'penalties_saved','saves','penalties_missed']

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(player_history_df[features], player_history_df[labels],
                                            test_size=0.2, random_state=42)

        # Create an empty dictionary to store model results for each label
        model_results = {}

        for label_to_predict in labels:
            # Train a separate model for each label
            model = train_model(X_train, y_train[label_to_predict])

            # Make predictions for player_future_fxt for the current label
            label_features = player_future_fxt[features]
            label_predictions = model.named_steps['model'].predict(label_features)

            # Create a DataFrame for the player's data
            player_data = player_history_df.copy()

            # Add predictions to player_future_fxt
            player_future_fxt[label_to_predict] = label_predictions

        # Append the combined data to the Don_Team DataFrame
        Don_Team = pd.concat([Don_Team, player_future_fxt], ignore_index=True)
        # Reset the index of Don_Team
        Don_Team.reset_index(drop=True, inplace=True)

        Don_Team = Don_Team.drop_duplicates(subset=['opponent_team','player_fixture_id'])
        Don_Team = Don_Team.loc[:,['team_id', 'web_name','Position','fixture_difficulty','opponent_team','player_fixture_id','goals_scored', 'assists', 'clean_sheets', 'goals_conceded','bonus', 'penalties_saved','saves','penalties_missed']]
        return Don_Team

    else:
        # Handle the case when data loading fails
        print(f"Data loading failed for player ID: {player_id}")




In [ ]:
# List of player IDs to process
player_ids = FPL_team.element  # Add the IDs you want to process



In [ ]:
Salah = process_player(308)

In [ ]:
Salah

## iv. Manager team predicted

In [ ]:
Manager_team = pd.DataFrame()

for i in player_ids:
  print("Player id: ", i)
  Manager_team = pd.concat([Manager_team, process_player(i)], ignore_index=True)


In [ ]:
Manager_team

In [ ]:
Manager_team

# 4. Points

In [116]:
#labels = ['goals_scored', 'assists', 'clean_sheets', 'goals_conceded','bonus', 'penalties_saved','saves','penalties_missed']

#Function for goal scored points
def goal_points(player_fixture_id):
  for idx, row in Manager_team.iterrows():
    if player_fixture_id == row['player_fixture_id']:
      if row['Position'] == 'FWD':
        return row['goals_scored'] * 4

      if row['Position'] == 'MID':
        return row['goals_scored'] * 5

      else:
        return row['goals_scored'] * 6



In [117]:
#Function for assists points
def assists_points(player_fixture_id):
  for idx, row in Manager_team.iterrows():
    if player_fixture_id == row['player_fixture_id']:
      return row['assists'] *3

In [118]:
#Function for clean_sheets scored points
def clean_sheets_points(player_fixture_id):
  for idx, row in Manager_team.iterrows():
    if player_fixture_id == row['player_fixture_id']:
      if row['Position'] == 'FWD':
        return row['clean_sheets'] * 0

      if row['Position'] == 'MID':
        return row['clean_sheets'] * 1

      else:
        return row['clean_sheets'] * 5



In [119]:
#Function for goals_conceded scored points
def goals_conceded_points(player_fixture_id):
  for idx, row in Manager_team.iterrows():
    if player_fixture_id == row['player_fixture_id']:
      if row['Position'] == 'FWD':
        return row['goals_conceded'] * 0

      if row['Position'] == 'MID':
        return row['goals_conceded'] * 0

      else:
        return row['goals_conceded'] * -1



In [120]:
#Function for bonus scored points
def bonus_points(player_fixture_id):
  for idx, row in Manager_team.iterrows():
    if player_fixture_id == row['player_fixture_id']:
      return row['bonus'] * 1

In [121]:
#Function for penalties_saved scored points
def penalties_saved_points(player_fixture_id):
  for idx, row in Manager_team.iterrows():
    if player_fixture_id == row['player_fixture_id']:
      return row['penalties_saved'] * 5



In [131]:
#Function for saves scored points
def saves_points(player_fixture_id):
  for idx, row in Manager_team.iterrows():
    if player_fixture_id == row['player_fixture_id']:
      if row['Position'] == 'DEF' or row['Position'] == 'GKP':
        return row['saves'] * 1/3


      else:
        return row['saves'] * 0



In [132]:
#Function for assists points
def penalties_missed_points(player_fixture_id):
  for idx, row in Manager_team.iterrows():
    if player_fixture_id == row['player_fixture_id']:
      return row['penalties_missed'] * -2

In [133]:
def total_points(player_fixture_id):
  return goal_points(player_fixture_id) + assists_points(player_fixture_id) + clean_sheets_points(player_fixture_id) + goals_conceded_points(player_fixture_id) + bonus_points(player_fixture_id) + penalties_saved_points(player_fixture_id) +saves_points(player_fixture_id) + penalties_missed_points(player_fixture_id)

In [134]:
Manager_team['Total_points'] = Manager_team['player_fixture_id'].apply(total_points)


In [135]:
Manager_team

,team_id,web_name,Position,fixture_difficulty,opponent_team,player_fixture_id,goals_scored,assists,clean_sheets,goals_conceded,bonus,penalties_saved,saves,penalties_missed,Total_points
0,8,Johnstone,GKP,3.0,17,230_10,-0.040898,-0.063630,0.437463,0.920295,0.595367,-0.004492,2.321247,-0.128403,2.434209
1,8,Johnstone,GKP,2.0,17,230_11,-0.054806,0.037198,0.252506,0.830201,0.524529,-0.001146,1.741501,-0.044190,1.402766
2,8,Johnstone,GKP,2.0,17,230_12,-0.059676,-0.029888,0.386570,0.964877,0.545300,-0.005056,1.926669,-0.097230,1.876956
3,18,Udogie,DEF,2.0,4,519_10,0.018809,0.184687,0.491942,0.838700,0.491338,0.041945,0.167631,-0.031886,3.108645
4,18,Udogie,DEF,3.0,4,519_11,-0.009448,0.155294,0.339142,0.863417,0.380130,0.001351,-0.144648,0.041666,1.496825
5,18,Udogie,DEF,2.0,4,519_12,0.018809,0.184687,0.491942,0.838700,0.491338,0.041945,0.167631,-0.031886,3.108645
6,2,Cash,DEF,2.0,15,36_10,0.145892,0.117559,0.309050,1.260648,0.429348,-0.083639,-0.012598,0.012529,1.494524
7,2,Cash,DEF,2.0,15,36_11,0.227585,0.346248,0.157664,1.303036,0.474705,0.086835,-0.061070,-0.048264,2.874590
8,2,Cash,DEF,2.0,15,36_12,0.145892,0.117559,0.309050,1.260648,0.429348,-0.083639,-0.012598,0.012529,1.494524
9,1,Saliba,DEF,2.0,16,20_10,-0.194500,0.079477,0.369141,0.744986,0.540762,0.027400,-0.046241,0.075163,0.684172
